# Ladda ned PDF regeringsuppdrag

Sidan https://www.regeringen.se/regeringsuppdrag/ anropar en [JSON-fil](https://www.regeringen.se/Filter/GetFilteredItems?filterType=Taxonomy&filterByType=FilterablePageBase&preFilteredCategories=1342&rootPageReference=0&page=1&pageSize=500&displayLimited=false&sortAlphabetically=False) dynamiskt vid vaje anrop. Genom att manipulera `pageSize` i URL:en kan man plocka ut 500 länkar åt gången och sedan skrapa länkens HTML (JSON-filen innehåller HTML som värde) och därifrån hämta länken till PDF-filen som man sedan laddar ned.

Not: Med tanke på att JSON-filen verkar vara generell för allt möjligt innehåll kan man nog ladda ned fler PDF-filer än bara regeringsuppdrag (se exempelvis `preFilteredCategories=1342` i URL:en). Har dock inte testat.

In [1]:
from urllib import request
from lxml import html
from bs4 import BeautifulSoup
import ssl

# skrapar html
def scrapewebpage(url):
    UseSSL = True  # Om du får SSLError, ändra detta till False.
    if UseSSL:
        web = request.urlopen(url)
    else:
        web = request.urlopen(url, context=ssl._create_unverified_context())
    if web.getcode() == 200:
        return(web.read())
    else:
        print("Error %s reading %s" % str(web.getcode()), url)

# Utforska

Se vad JSON-filen innehåller för något smått och gott.

In [2]:
# skrapa json-fil
data = scrapewebpage("https://www.regeringen.se/Filter/GetFilteredItems?filterType=Taxonomy&filterByType=FilterablePageBase&preFilteredCategories=1342&rootPageReference=0&page=1&pageSize=500&displayLimited=false&sortAlphabetically=False")

In [3]:
# läs in json
import json
j = json.loads(data)

# nycklar i json-datan
for key in j.keys():
    print(key)

Message
TotalCount


In [4]:
# regeringsuppdrag totalt
j["TotalCount"]

2190

In [5]:
# plocka ut alla länkar på sidan från j["Message"] som innehåller html
soup = BeautifulSoup(j["Message"], "lxml-xml")
links = soup.find_all("a", "readmore")
len(links)

500

In [6]:
# kolla hur url:erna till regeringsuppdragen ser ut
url = links[0].get("href")
url

'/regeringsuppdrag/2018/06/uppdrag-till-polismyndigheten-och-aklagarmyndigheten-om-forbattrad-hantering-av-bidragsbrott/'

In [7]:
# hämta egeringsuppdragssidornas html
html = scrapewebpage("https://regeringen.se" + url)

In [8]:
# kolla var ".pdf" förekommer
pos = str(html).lower().find(".pdf")
html[pos-1000:pos+1000]

b'  <ul class="list--Block--icons" >\r\n\r\n                                    <li>            <a href="/49cf68/contentassets/5deecb383435428b97154777223c0017/uppdrag-till-polismyndigheten-och-aklagarmyndigheten-om-forbattrad-hantering-av-bidragsbrott.pdf">Uppdrag till Polismyndigheten och &#197;klagarmyndigheten om f&#246;rb&#228;ttrad hantering av bidragsbrott (pdf 86 kB)</a>\r\n</li>\r\n                    </ul>\r\n\r\n\r\n                \r\n\r\n\r\n                \r\n\r\n\r\n                \r\n                \r\n                \r\n                \r\n                \r\n                <div class="has-wordExplanation">\r\n                    <p>Polismyndigheten och \xc3\x85klagarmyndigheten ska samr\xc3\xa5da med de akt\xc3\xb6rer som \xc3\xa4r relevanta f\xc3\xb6r att \xc3\xa5stadkomma en effektivare och f\xc3\xb6rb\xc3\xa4ttrad hantering av bidragsbrott, framf\xc3\xb6r allt Ekobrottsmyndigheten, F\xc3\xb6rs\xc3\xa4kringskassan och \xc3\xb6vriga utbetalande myndigheter.</p>\

# Försök parsa PDF-fil och metadata från HMTL

Nu när vi vet ungefär vad JSON-filen innehåller och hur HTML-sidorna är strukturerade, skapa funktioner för att hantera pdf-filernas namn, hämta ut metadata m.m.

In [9]:
# skapa några helper-funktioner
import urllib.request
from urllib.parse import urlparse
import os

# ta ut pdf-filnamnet (antar att första träffen på ".pdf" är rätt)
def get_pdf(pdf_soup):
    for a in pdf_soup.find_all("a"):
        if a != None:
            url = a.get("href")
            if url.lower().find(".pdf") > 0:
                return("https://regeringen.se" + url)
    # okej, hittade inget .pdf - gör då några desperata gissningar var pdf-filen finns
    url = [div.find("a") for div in pdf_soup.findAll("", "col-1")]
    if url != None:
        return("https://regeringen.se" + url[0].get("href"))
    # hittade verkligen inget, ge upp och gråt
    return("")

# ta ut pdf-filnamnet från url
def get_filename(url):
    if url.find("/"):
        return(url.rsplit("/", 1)[1])

# ladda ned pdf
def download_pdf(url):
    urllib.request.urlretrieve(url, get_filename(url))
    
# ta ut övrig metadata från regeringsuppdragets sida (titel, diarienummer, beskrivning, datum etc)
def get_metadata(url, pdf_soup):
    title = ""
    dnr = ""
    ingress = ""
    beskrivning = ""
    pub = ""
    h1 = pdf_soup.find("h1").get_text().strip()
    try:
        dnr = h1.split("\n")[1].strip()
        title = h1.split("\n")[0].strip()
    except:
        title = h1
        dnr = ""
    try:
        ingress = pdf_soup.find("", "ingress has-wordExplanation").get_text().strip()
    except:
        pass
    try:
        explanation = pdf_soup.find("", "has-wordExplanation").get_text().strip()
    except:
        pass
    try:
        pub = pdf_soup.find("time").get("datetime")
    except:
        pass
    return({"titel": title, "dnr": dnr, "ingress": ingress, "beskrivning": explanation,  "pub": pub, 
            "pdf": get_filename(get_pdf(pdf_soup)), "url": url})

In [13]:
# kolla att det verkar funka på ett regeringsuppdrag
url = "https://regeringen.se/regeringsuppdrag/2018/06/uppdrag-om-saker-och-effektiv-tillgang-till-grunddata/"
html = scrapewebpage(url)
pdf_soup = BeautifulSoup(html, "lxml-xml")
get_metadata(url, pdf_soup)

{'titel': 'Uppdrag om säker och effektiv tillgång till grunddata',
 'dnr': '',
 'ingress': '',
 'beskrivning': 'Regeringen gerBolagsverket, Lantmäteriet och Skatteverket i uppdrag att tillsammans lämna förslag som syftar till att skapa en säker och effektiv till\xadgång till grunddata, genom att bl.a. tydliggöra ansvaret för och öka standardi\xadseringen av sådan data.\nUppdraget ska slutredovisas till regeringen senast den 30 april 2019.',
 'pub': '07 juni 2018',
 'pdf': 'uppdrag-om-saker-och-effektiv-tillgang-till-grunddata.pdf',
 'url': 'https://regeringen.se/regeringsuppdrag/2018/06/uppdrag-om-saker-och-effektiv-tillgang-till-grunddata/'}

In [12]:
# vissa regeringsuppdrag har inget .pdf i slutet av filnamnet, som den här,
# kolla att den kan parsa ut pdf-filen ändå
url = "https://regeringen.se/regeringsuppdrag/2018/06/uppdrag-till-arbetsformedlingen-om-en-modell-for-upphandlad-matchning-till-etableringsjobb/"
html = scrapewebpage(url)
pdf_soup = BeautifulSoup(html, "lxml-xml")
get_metadata(url, pdf_soup)

{'titel': 'Uppdrag till Arbetsförmedlingen om en modell för upphandlad matchning till etableringsjobb',
 'dnr': 'Diarienummer: A2018/01212/A',
 'ingress': 'Regeringen uppdrar åt Arbetsförmedlingen att analysera förutsättningarna för och utreda den närmare utformningen av en modell för upphandlad matchning till etableringsjobb. Uppdraget ska genomföras i samråd med LO, Unionen och Svenskt Näringsliv.',
 'beskrivning': 'Arbetsförmedlingen ska analysera hur tjänsten Stöd och matchning kan utvecklas och hur tjänsten kan användas tillsammans med andra insatser för att skapa en effektiv matchning till etableringsjobb. Arbetsförmedlingen ska utifrån detta perspektiv analysera dimensioneringen av tjänsten.\nUtifrån analysen ska Arbetsförmedlingen redovisa hur utformningen av en upphandlad modell för matchning till etableringsjobb skulle kunna se ut. Om det bedöms finnas behov av författningsändringar för att kunna utforma en effektiv modell för upphandlad matchning till etableringsjobb ska såd

# Skapa sqlite3 databas

Någonstans måste vi ju spara allt.

In [14]:
import sqlite3 as lite
con = lite.connect("regeringsuppdrag.db")
cur = con.cursor()    

# skapa databas om den inte redan finns
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
counter = cur.fetchall()
if len(counter) < 2:
    cur.execute("CREATE TABLE regeringsuppdrag (id INTEGER PRIMARY KEY AUTOINCREMENT, titel TEXT, dnr TEXT, ingress TEXT, beskrivning TEXT, pub TEXT, pdf TEXT, url TEXT)")
    
# dumpa json till sqlite
def write_metadata(j):
    with con:
        cur = con.cursor()
        cur.execute("INSERT INTO regeringsuppdrag (titel, dnr, ingress, beskrivning, pub, pdf, url) VALUES (?, ?, ?, ?, ?, ?, ?)",
                    (j["titel"], j["dnr"], j["ingress"], j["beskrivning"], j["pub"], j["pdf"], j["url"]))

# Skrapa allt

Nu verkar allt fungera att hämta.

Detta är huvudfunktion för att skrapa alla pdf:er. Notera att det är 1,5 sekunders fördröjning:

In [15]:
import csv
import time
import sys
import math
i = 0
errors = 0
        
# 2190 regeringsuppdrag / 500 per sida = 5 sidor,
# vi behöver med andra ord köra for-loopen 5 gånger
for page in range(1, math.ceil(int(j["TotalCount"]) / 500) + 1): 
    print("Sida {0}".format(page))
    # skrapa 500 resultat åt gången och ta ut alla (förhoppningsvis) 500 länkar
    data = scrapewebpage("https://www.regeringen.se/Filter/GetFilteredItems?filterType=Taxonomy&filterByType=FilterablePageBase&preFilteredCategories=1342&rootPageReference=0&page=" + str(page) + "&pageSize=500&displayLimited=false&sortAlphabetically=False")
    j = json.loads(data)    
    soup = BeautifulSoup(j["Message"], "lxml-xml")
    links = soup.find_all("a", "readmore")
    
    # skrapa länk för länk med regeringsuppdrag
    print("Laddar ned pdf", end=" ")
    for a in links:
        if a != None:
            url = a.get("href")
            try:
                # ta ut varje sida med regeringsuppdrag & ladda ned pdf
                i += 1
                html = scrapewebpage("https://regeringen.se" + url)
                pdf_soup = BeautifulSoup(html, "lxml-xml")
                pdf_url = get_pdf(pdf_soup)
                if pdf_url != "":
                    #print("Laddar ned https://regeringen.se" + pdf_url)
                    print(i, end=" ")
                    write_metadata(get_metadata(url, pdf_soup))
                    download_pdf(pdf_url)
                    time.sleep(1.5) # Fördröjning för att vara snäll mot regeringen (maskinerna kommer ta över världen ändå en vacker dag)
                else:
                    print()
                    print("hittade ingen pdf: https://regeringen.se" + url)
            except KeyboardInterrupt:
                con.close()
                sys.exit()
            except:
                errors += 1
                print()
                print("Fel: https://regeringen.se" + url + " " + str(sys.exc_info()[0]))
                  
con.close()
print()
print()
print("Klart, {0} pdf:er nedladdat, {1} fel".format(i, errors))

Sida 1
Laddar ned pdf 1 2 3 4 5 6 

SystemExit: 

C:\Users\Peter\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
